<a href="https://colab.research.google.com/github/AllanSoto/DASHBOARD-RESUELVA-CR-2025./blob/main/DASHBOARD_RESUELVA_CR_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install dash dash-bootstrap-components plotly pyngrok --quiet

# DASHBOARD RESUELVA CR 2025.ipynb

import pandas as pd
import dash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
from google.colab import drive
from pyngrok import ngrok

# ngrok authtoken
ngrok.set_auth_token("2vgkAdqK5LMidWrmW9DIqHzBgWh_5M5QftoWLqRHg9z6xWqX9")
drive.mount('/content/drive')

# Cargar datos
csv_path = '/content/drive/MyDrive/Colab Notebooks/Libro1.csv'
df = pd.read_csv(csv_path, sep=';')

# Preprocesamiento
df['fecha_registro'] = pd.to_datetime(df['fecha_registro'], dayfirst=True, errors='coerce')
for col in ['deuda','recupero','promesa_pago']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df['año']  = df['fecha_registro'].dt.year
df['mes']  = df['fecha_registro'].dt.month
df['día']  = df['fecha_registro'].dt.day

# Filtrar solo 2025
df_2025 = df[df['año'] == 2025]

# Métricas 2025
total_recupero = df_2025['recupero'].sum()
total_promesa  = df_2025['promesa_pago'].sum()
porcentaje_rec  = (total_recupero / total_promesa * 100) if total_promesa > 0 else 0
pendiente_pago  = total_promesa - total_recupero

# Colores y estilos
bg_color      = '#1e1e1e'
font_color    = 'white'
color_primary = 'lightskyblue'
color_success = 'lightgreen'
card_style    = {'backgroundColor': bg_color, 'border': '1px solid #444', 'color': font_color}

# Gauge: Recupero vs Promesa
fig_gauge = go.Figure(go.Indicator(
    mode="gauge+number",
    value=total_recupero,
    number={'prefix':'₡','font':{'color':font_color}},
    gauge={
        'axis':{'range':[0, total_promesa]},
        'bar':{'color':color_primary},
        'bgcolor':bg_color,
        'steps':[{'range':[0,total_promesa], 'color':'#333333'}]
    }
))
fig_gauge.update_layout(
    title={'text':'Recupero vs Promesa 2025','font':{'color':font_color}},
    paper_bgcolor=bg_color, font_color=font_color
)

# KPI Cards
card_porcentaje = dbc.Card([
    dbc.CardHeader("% Recuperado", style={'backgroundColor':'#222','color':font_color}),
    dbc.CardBody(html.H3(f"{porcentaje_rec:.2f}%", style={'color':color_success}))
], style=card_style)

card_pendiente = dbc.Card([
    dbc.CardHeader("Pendiente Pago", style={'backgroundColor':'#222','color':font_color}),
    dbc.CardBody(html.H3(f"₡{pendiente_pago:,.0f}", style={'color':color_primary}))
], style=card_style)

tarjeta_recupero = dbc.Card([
    dbc.CardHeader("Total Recuperado", style={'backgroundColor':'#222','color':font_color}),
    dbc.CardBody(html.H3(f"₡{total_recupero:,.0f}", style={'color':color_success}))
], style=card_style)

tarjeta_promesas = dbc.Card([
    dbc.CardHeader("Total Promesas", style={'backgroundColor':'#222','color':font_color}),
    dbc.CardBody(html.H3(f"₡{total_promesa:,.0f}", style={'color':color_primary}))
], style=card_style)

# Gráfica mensual: Recupero vs Promesa
df_mes = df_2025.groupby('mes')[['recupero','promesa_pago']].sum().reset_index()
fig_mes = px.bar(
    df_mes, x='mes', y=['recupero','promesa_pago'], barmode='group',
    labels={'value':'Monto','mes':'Mes'}, title='Recupero vs Promesa por Mes 2025',
    color_discrete_sequence=[color_success, color_primary]
)
fig_mes.update_layout(plot_bgcolor=bg_color, paper_bgcolor=bg_color, font_color=font_color)

# Gráfica diaria: Recupero
df_dia = df_2025.groupby('fecha_registro')['recupero'].sum().reset_index()
fig_dia = px.line(
    df_dia, x='fecha_registro', y='recupero',
    labels={'recupero':'Recupero','fecha_registro':'Fecha'}, title='Recupero Diario 2025',
    markers=True
)
fig_dia.update_traces(line=dict(color=color_success), marker=dict(color=color_success))
fig_dia.update_layout(plot_bgcolor=bg_color, paper_bgcolor=bg_color, font_color=font_color)

# Donut: Recupero por Subcategoría
df_sub = df_2025.groupby('subcategoria')['recupero'].sum().reset_index()
fig_donut = px.pie(
    df_sub, names='subcategoria', values='recupero', hole=0.5,
    title='Recupero por Subcategoría 2025',
    color_discrete_sequence=px.colors.sequential.Plasma
)
fig_donut.update_layout(plot_bgcolor=bg_color, paper_bgcolor=bg_color, font_color=font_color)

# Tabla detalle subcategoría
df_tabla = df_2025.groupby('subcategoria').agg(
    Total_Presupuesto=('promesa_pago','sum'),
    Total_Recupero=('recupero','sum')
).reset_index()
df_tabla['Saldo'] = df_tabla['Total_Presupuesto'] - df_tabla['Total_Recupero']
df_tabla['% Recuperado'] = (df_tabla['Total_Recupero']/df_tabla['Total_Presupuesto']*100).round(2)

table = dash_table.DataTable(
    data=df_tabla.to_dict('records'),
    columns=[{"name":c, "id":c} for c in df_tabla.columns],
    style_header={'backgroundColor':'#222', 'color':font_color},
    style_cell={'backgroundColor':bg_color, 'color':font_color, 'textAlign':'center'},
    style_data_conditional=[
        {'if':{'column_id':'% Recuperado','filter_query':'{% Recuperado} > 80'}, 'backgroundColor':'#004d00'},
        {'if':{'column_id':'% Recuperado','filter_query':'{% Recuperado} < 50'}, 'backgroundColor':'#660000'}
    ],
    page_size=10,
)

# Inicializar App
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

# Meta-refresh cada 10 min
app.index_string = '''
<!DOCTYPE html>
<html>
  <head>
    {%metas%}
    <title>RESUELVA CR 2025</title>
    {%favicon%}
    {%css%}
    <meta http-equiv="refresh" content="600">
  </head>
  <body>
    {%app_entry%}
    <footer>
      {%config%}
      {%scripts%}
      {%renderer%}
    </footer>
  </body>
</html>
'''

# Layout
app.layout = dbc.Container(
    fluid=True,
    style={'backgroundColor':bg_color},
    children=[
        dbc.Row(dbc.Col(html.H1("RESUELVA CR 2025", className="text-center text-light py-3"))),
        dbc.Row([
            dbc.Col(dcc.Graph(figure=fig_gauge),   md=4),
            dbc.Col(card_porcentaje,               md=2),
            dbc.Col(card_pendiente,                md=2),
            dbc.Col(tarjeta_recupero,              md=2),
            dbc.Col(tarjeta_promesas,              md=2),
        ], className="mb-4"),
        dbc.Row([
            dbc.Col(dcc.Graph(figure=fig_mes),     md=6),
            dbc.Col(dcc.Graph(figure=fig_dia),     md=6),
        ], className="mb-4"),
        dbc.Row([
            dbc.Col(dcc.Graph(figure=fig_donut),   md=6),
            dbc.Col(table,                         md=6),
        ], className="mb-4"),
    ]
)

# Ejecutar y exponer públicamente con ngrok
if __name__=='__main__':
    public_url = ngrok.connect(8050)
    print('Tu dashboard está disponible en:', public_url)
    app.run(debug=False, port=8050)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Tu dashboard está disponible en: NgrokTunnel: "https://9f9e-34-69-98-132.ngrok-free.app" -> "http://localhost:8050"


<IPython.core.display.Javascript object>